In [ ]:
import numpy as np

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.NoiseModel import construct_noise_model_from_hardware_model
from LogicalQ.Estimators import compute_effective_threshold
from LogicalQ.Benchmarks import mirror_benchmarking
from LogicalQ.Experiments import execute_circuits, qec_cycle_efficiency_experiment
from LogicalQ.Analysis import qec_cycle_efficiency_scatter

## Compute the effective threshold

In [ ]:
_, _, effective_threshold = compute_effective_threshold(
    hardware_models_Quantinuum["H2-1"],
    **steane_code
)

## Comparing single-constraint models

### Prepare experiment inputs

In [ ]:
qc = mirror_benchmarking(n_qubits=1, circuit_length=8, seed=1111)
qc.draw("mpl")

In [ ]:
constraint_scan_key_list = [
    "cost_ops_1q",
    "cost_ops_pauli_1q",
    "cost_ops_clifford_1q",
    "cost_ops_nonclifford_1q",
]

constraint_scan_values = np.linspace(effective_threshold/20, effective_threshold, 20)

In [ ]:
data_list = []
for constraint_scan_key in constraint_scan_key_list:
    data = qec_cycle_efficiency_experiment(
        circuit_input=qc,
        qecc=steane_code,
        constraint_scan_keys=["effective_threshold", constraint_scan_key],
        constraint_scan_val_lists=[[effective_threshold], constraint_scan_values],
        backend="aer_simulator",
        hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, noise_model=None,
        method="statevector", shots=1E3, with_mp=False,
        save_dir=None, save_filename=None
    )
    data_list.append(data)

In [ ]:
for data, constraint_scan_key in zip(data_list, constraint_scan_key_list):
    qec_cycle_efficiency_scatter(data, scan_keys=[constraint_scan_key])

In [ ]:
qec_cycle_indices_list = [(list(datum["constraint_model"].values())[1], datum["qec_cycle_indices"]) for datum in data_list[0][0]["results"]]
print(qec_cycle_indices_list)

In [ ]:
lqc = data_list[0][0]["logical_circuit"]
lqc.insert_qec_cycles(qec_cycle_indices=qec_cycle_indices_list[-1][1])
lqc.draw("mpl")